# X-ray emission spectrscopy (XES)


In this exercise, you will use emission data measured on Cr<sub>2</sub>O<sub>3</sub>. The first steps are the same as in the case of the XAS data reduction, i.e. locate the data on disk and visualize it. The particularity of this data reduction, is that the final spectrum will be reconstructed from three scans performed using different energy steps.

<figure>
  <img src="assets/xes.png" alt="XAS" style="width:60%">
  <figcaption style="text-align: center; font-style: italic">J.K. Kowalska et al., Israel Journal of Chemistry 56, 803 (2016).</figcaption>
</figure>

## Import the required packages

In [ ]:
%matplotlib ipympl

import sys
import logging

import numpy as np
import matplotlib.pyplot as plt
import silx.io.h5py_utils

plt.rcParams["figure.figsize"] = (6, 3.4)

## Accessing the data

As you did in the XAS exercise, you must first locate the data on disk and load the information from various counters. For XES, the data is stored in the same file as before, but the energy counter, i.e., the X-axis, changes and is now `xes_en`. The signal and the monitor remain as before.

Load and plot the data from scans 4, 5, and 6.

In [ ]:
filename = "experimental_data/ihch1515/id26/Cr2O3_new/Cr2O3_new_0003/Cr2O3_new_0003.h5"

# Define a list for the scan indices.
##################
# YOUR CODE HERE #
scan_ids = [4, 5, 6]
##################

fig, ax = plt.subplots()

with silx.io.h5py_utils.File(filename) as sf:
    for scan_id in scan_ids:
        scan_name = f"{scan_id}.1"
        x = sf[f"{scan_name}/measurement/xes_en"][()]
        signal = sf[f"{scan_name}/measurement/det_dtc_apd"][()]
        monitor = sf[f"{scan_name}/measurement/I02"][()]
        ax.plot(x, signal / monitor, ".", label=f"{scan_id}")
ax.legend()

## Merging the spectra

Notice that, in this case, the signals appear at different energy positions. Because the XES measurements can have long acquisition times, it is common to sample regions of the spectrum differently. 

Start by storing the needed arrays in a list to avoid always reading them from the HDF5 file. 

In [ ]:
scans = []
with silx.io.h5py_utils.File(filename) as sf:
    for scan_id in scan_ids:
        scan_name = f"{scan_id}.1"
        x = sf[f"{scan_name}/measurement/xes_en"][()]
        signal = sf[f"{scan_name}/measurement/det_dtc_apd"][()]
        monitor = sf[f"{scan_name}/measurement/I02"][()]
        scans.append([x, signal, monitor])        

Create temporary containers to hold the x, signal, and monitor data from the three scans.

In [ ]:
x_tmp, signal_tmp, monitor_tmp = [], [], []

for x, signal, monitor in scans:
    x_tmp.extend(x)
    signal_tmp.extend(signal)
    monitor_tmp.extend(monitor)

Find the limits and step size of the new X-axis.

In [ ]:
xmin, xmax = np.inf, -np.inf
for x, signal, monitor in scans:
    x = np.sort(x)
    # Calculate the new values for the xmin and xmax.
    ##################
    # YOUR CODE HERE #    
    xmin = min(xmin, x.min())
    xmax = max(xmax, x.max())
    ##################
    xstep = min(np.diff(x))

Create the new X-axis array using the `np.arange` function. You can check the function documentation using `np.arange?`.

In [ ]:
##################
# YOUR CODE HERE # 
xn = np.arange(xmin, xmax, xstep)
##################
# Print the first 5 elements of the array.
print(xn[:5])

Calculate and plot the merged signal.

In [ ]:
# Convert the temporary lists to Numpy arrays.
x = np.array(x_tmp)
signal = np.array(signal_tmp)
monitor = np.array(monitor_tmp)

# Find the indices that sort the x values.
ids = np.argsort(x_tmp)

# Sort all arrays using these indices. The interpolation 
# can be wrong if the the arrays are not sorted.
x = x[ids]
singal = signal[ids]
monitor = monitor[ids]

# Normalize with the monitor and interpolate the resulting signal.
##################
# YOUR CODE HERE # 
signal = signal / monitor
signal = np.interp(xn, x, signal)
##################

# Plot the merged signal.
fig, ax = plt.subplots(figsize=(6, 3.4))  
ax.plot(xn, signal)

## Plot the data with log scaling on the Y-axis

Look for the plotting function in the Matplotlib library.

In [ ]:
fig, ax = plt.subplots(figsize=(7, 3.2))

##################
# YOUR CODE HERE # 
ax.semilogy(xn, signal, "k-")    
##################

ax.set_xlabel("Emission energy (eV)")
ax.set_ylabel("Intensity (arb. units)")

plt.tight_layout()

## Background subtraction for the valence-to-core (vtc) XES

<figure>
  <img src="assets/xes_semilogy.png" alt="XES semilog Y-axis" style="width:60%">
  <figcaption style="text-align: center; font-style: italic">E. Gallo and P. Glatzel, Advanced Materials 26, 7730 (2014).</figcaption>
</figure>